In [1]:
import os
import glob
import re
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from collections import Counter

# nltk.download('stopwords')
# ================================
# Define Classes for Speaker and Speech
# ================================
    
class Speaker:
    def __init__(self, speakerid, last_name, first_name, chamber, state, gender, party, district, nonvoting):
        self.speakerid = speakerid
        self.last_name = last_name
        self.first_name = first_name
        self.chamber = chamber
        self.state = state
        self.district = district
        self.party = party
        self.nonvoting = nonvoting
        self.speeches = []  # Will hold speech_id's

    def add_speech(self, speech_id):
        self.speeches.append(speech_id)

class Speech:
    def __init__(self, speech_id, chamber, date, word_count, text):
        self.speech_id = speech_id
        # Convert date string to datetime (assumes YYYY-MM-DD format; adjust if needed)
        self.date = datetime.strptime(date, "%Y%m%d")
        self.chamber = chamber
        self.word_count = int(word_count)
        self.text = text
        # These will be computed later
        self.D_sim_av = 0.0
        self.D_sim = 0.0  
        self.R_sim_av = 0.0
        self.R_sim = 0.0  


# ================================
# Parsing Functions
# ================================

def parse_speaker_map(filename):
    """
    Reads a speaker map file that may have either:
      - 11 fields: speakerid|speech_id|lastname|firstname|chamber|state|gender|party|district||nonvoting
      - 10 fields: speakerid|speech_id|lastname|firstname|chamber|state|gender|party|district|nonvoting
    and returns a dict mapping speakerid -> Speaker object.
    """
    speakers = {}
    with open(filename, 'r', encoding='latin-1') as f:
        next(f)  # Skip header/sample line
        for line in f:
            line = line.rstrip('\n')
            if not line:
                continue
            parts = line.split('|')
            # Ensure we have at least 10 parts (positions 0-9)
            if len(parts) < 10:
                print(f"Skipping unexpected format: {line}")
                continue

            speakerid = parts[0]
            speech_id = parts[1]
            lastname = parts[2]
            firstname = parts[3]
            chamber = parts[4]
            state = parts[5]
            gender = parts[6]
            party = parts[7]
            district = parts[8]
            if not district:
                district = "at-large"

            # Check if there is an extra empty field before nonvoting
            if len(parts) == 11:
                nonvoting = parts[10]
            else:
                nonvoting = parts[9]

            # Now create the Speaker object
            if speakerid not in speakers:
                speakers[speakerid] = Speaker(speakerid, lastname, firstname, chamber, state, gender, party, district, nonvoting)
            speakers[speakerid].add_speech(speech_id)
    return speakers


def parse_speeches(filename):
    """
    Reads a speeches file with format:
    speech_id|speech
    (Note: speeches can be multiline but here we assume each entry is a single record separated by newline.
    If your files have a more complex format, you may need to adjust the parsing logic.)
    Returns a dict mapping speech_id -> speech text.
    """
    speeches = {}
    with open(filename, 'r', encoding='utf-8') as f:
        next(f)  # Skip header/sample line
        for line in f:
            line = line.strip()
            if line:
                # Split only on the first delimiter (in case the text contains a "|")
                speech_id, speech_text = line.split('|', 1)
                speeches[speech_id] = speech_text
    return speeches

def parse_description(filename):
    """
    Reads a description file with format:
    speech_id|chamber|date|number_within_file|speaker|first_name|last_name|state|gender|line_start|line_end|file|char_count|word_count
    Returns a dict mapping speech_id -> dictionary with description info.
    """
    descriptions = {}
    with open(filename, 'r', encoding='latin-1') as f:
        next(f)  # Skip header/sample line
        for line in f:
            line = line.strip()
            if line:
                parts = line.split('|')
                speech_id = parts[0]
                chamber = parts[1]
                date = parts[2]
                # parts[13] should be word_count (if indexing starts at 0)
                word_count = parts[13]
                descriptions[speech_id] = {'chamber': chamber, 'date': date, 'word_count': word_count}
    return descriptions

def create_speech_objects(speeches_dict, descriptions):
    """
    Combines the raw speech text (from speeches_dict) with the description info.
    Returns a dict mapping speech_id -> Speech object.
    """
    speech_objects = {}
    for speech_id, text in speeches_dict.items():
        if speech_id in descriptions:
            info = descriptions[speech_id]
            try:
                speech_obj = Speech(speech_id, info['chamber'], info['date'], info['word_count'], text)
                speech_objects[speech_id] = speech_obj
            except Exception as e:
                print(f"Error creating Speech object for speech_id {speech_id}: {e}")
        else:
            print(f"Warning: No description found for speech_id {speech_id}")
    return speech_objects

def get_top_words(bigram_file, n):
    """
    Reads a bigram score file with format:
    phrase|score
    Returns two lists:
      - democrat_words: top n phrases with the most negative scores (associated with Democrats)
      - republican_words: top n phrases with the most positive scores (associated with Republicans)
    """
    phrases = []
    with open(bigram_file, 'r', encoding='latin-1') as f:
        next(f)  # Skip header/sample line.
        for line in f:
            line = line.strip()
            if line:
                phrase, score_str = line.split('|')
                score = float(score_str)
                # Convert spaces to underscores and lowercase the phrase.
                phrase = phrase.lower().replace(" ", "_")
                phrases.append((phrase, score))
    # Sort for Democrat and Republican words.
    phrases_sorted = sorted(phrases, key=lambda x: x[1])
    democrat_words = [phrase for phrase, score in phrases_sorted[:n]]
    phrases_sorted_desc = sorted(phrases, key=lambda x: -x[1])
    republican_words = [phrase for phrase, score in phrases_sorted_desc[:n]]
    return democrat_words, republican_words

# ================================
# Word2Vec Functions
# ================================

def build_corpus(speech_objects):
    """
    Builds a corpus (list of token lists) from the text of all Speech objects.
    For a better analysis you might consider more sophisticated tokenization.
    """
    corpus = []
    for speech in tqdm(speech_objects.values(), desc="Building corpus"):
        tokens = speech.text.split()  # simple whitespace tokenization; consider using nltk.word_tokenize if needed
        corpus.append(tokens)
    return corpus

def train_word2vec(corpus, vector_size=100, window=5, min_count=5, workers=4):
    """
    Trains a Word2Vec model on the provided corpus.
    """
    model = Word2Vec(sentences=corpus, vector_size=vector_size, window=window, min_count=min_count, workers=workers)
    return model

def average_vector(model, words):
    """
    Given a list of words and a trained Word2Vec model,
    returns the average vector for all words that appear in the model's vocabulary.
    """
    vecs = []
    for word in words:
        if word in model.wv:
            vecs.append(model.wv[word])
        else:
            # Optionally: you might want to try a lowercased version or other normalization
            pass
    if vecs:
        avg_vec = np.mean(vecs, axis=0)
    else:
        avg_vec = np.zeros(model.vector_size)
    return avg_vec


def compute_similarity_for_speech(speech, model, target_vec, bigram_transformer, p_threshold):
    # Tokenize and transform to bigrams.
    tokens = speech.text.split()  # Use your preferred tokenizer.
    tokens = bigram_transformer[tokens]
    
    sim_scores = []
    for token in tokens:
        if token in model.wv:
            word_vec = model.wv[token]
            # Compute cosine similarity.
            cos_sim = np.dot(word_vec, target_vec) / (np.linalg.norm(word_vec) * np.linalg.norm(target_vec) + 1e-9)
            sim_scores.append(cos_sim)
    if len(sim_scores) == 0:
        return 0.0, 0.0, sim_scores  # Return zeros if no tokens found.

    # Sort the similarity scores in descending order.
    sim_scores.sort(reverse=True)
    
    # Compute the threshold value (p-th percentile).
    threshold_value = np.percentile(sim_scores, p_threshold)
    
    # Only include tokens with similarity >= threshold.
    n_top = max(1, int(len(sim_scores) * p_threshold))
    top_scores = sim_scores[:n_top]
    thresh_avg = sum(top_scores) / len(top_scores)

    # Raw average: average of all token similarity scores.
    raw_avg = np.mean(sim_scores)

    return raw_avg, thresh_avg, sim_scores
    
def compute_similarity_for_phrases(model, input_token, target_token):

        word_vec = model.wv[input_token]
        target_vec = model.wv[target_token]
        cos_sim = np.dot(word_vec, target_vec) / (np.linalg.norm(word_vec) * np.linalg.norm(target_vec) + 1e-9)
        return cos_sim


# ================================
# Panel Dataset Aggregation
# ================================

def create_panel_dataframe(speech_objects, speech_to_speaker):
    """
    Constructs a DataFrame with one row per speech that includes:
      - date of speech
      - district (combination of state and district)
      - D_sim, R_sim scores
      - word_count
    The speech_to_speaker is a dict mapping speech_id -> Speaker object.
    """
    data = []
    for speech in tqdm(speech_objects.values(), desc="Creating Panel Dataframe"):
        if speech.speech_id in speech_to_speaker:
            speaker = speech_to_speaker[speech.speech_id]
            # Construct a district identifier (e.g., "NY-12")
            district = f"{speaker.state}-{speaker.district}"
            data.append({
                "date": speech.date,
                "district": district,
                "D_sim": speech.D_sim,
                "R_sim": speech.R_sim,
                "word_count": speech.word_count
            })
        else:
            print(f"Warning: Speaker not found for speech_id {speech.speech_id}")
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    return df

def compute_weighted_rolling_average(df, window='60D'):
    """
    For each congressional district, compute a weighted rolling average over a window (default 60 days)
    of the D_sim and R_sim scores using word_count as weights.
    Returns a dictionary mapping district -> DataFrame with the rolling averages.
    """
    panel_data = {}
    for district, group in tqdm(df.groupby("district"), desc="Creating Rolling Averages For Districts"):
        # Ensure sorting by date and set the date as index
        group = group.sort_values("date").set_index("date")
        # Define a lambda that computes a weighted average from a Series x,
        # using the corresponding word_count values from the same index.
        # Here we use group.loc[x.index, 'word_count'] as weights.
        group['D_sim_avg'] = group['D_sim'].rolling(window, min_periods=1).apply(
            lambda x: np.average(x, weights=group.loc[x.index, 'word_count']),
            raw=False
        )
        group['R_sim_avg'] = group['R_sim'].rolling(window, min_periods=1).apply(
            lambda x: np.average(x, weights=group.loc[x.index, 'word_count']),
            raw=False
        )
        panel_data[district] = group
    return panel_data

# ================================
# Main Processing Pipeline
# ================================

# Dictionaries to hold all objects across sessions
all_speakers = {}
all_speech_objects = {}
speech_to_speaker = {}  # Maps speech_id to Speaker object

# We will also accumulate bigram words across sessions
democrat_words_all = []
republican_words_all = []

#Global dataframe rows
rows = []

#Hyperparamter: number of top words used in analysis
n = 100

#Parameter: threshold percentile as decimal (for top decile, use 0.1).
p_threshold = 0.1

#Collection of Similarity Scores
sim_scores = []

#Session dataframe array
session_dfs = []

# Process sessions 097 to 114 (inclusive)
for session in tqdm(range(97, 115), desc="Processing sessions"):
    session_str = f"{session:03d}"  # e.g., "097", "098", ... "114"
    
    # Define filenames (adjust paths if needed)
    speaker_map_file = f"C:\\Users\\Maxfield Evers\\Desktop\\Thesis\\Data\\gentzkow_work\\hein-daily\\hein-daily\\{session_str}_SpeakerMap.txt"
    speeches_file = f"C:\\Users\\Maxfield Evers\\Desktop\\Thesis\\Data\\gentzkow_work\\hein-daily\\hein-daily\\cleaned_speeches_{session_str}.txt"
    descr_file = f"C:\\Users\\Maxfield Evers\\Desktop\\Thesis\\Data\\gentzkow_work\\hein-daily\\hein-daily\\descr_{session_str}.txt"
    
    # Parse speaker map and update our global speaker dictionary
    if os.path.exists(speaker_map_file):
        speakers = parse_speaker_map(speaker_map_file)
        all_speakers.update(speakers)
        # Create a mapping from speech_id to speaker object
        for sp in speakers.values():
            for s_id in sp.speeches:
                speech_to_speaker[s_id] = sp
    else:
        print(f"File not found: {speaker_map_file}")
    
    # Parse speeches and their descriptions; then create Speech objects
    if os.path.exists(speeches_file) and os.path.exists(descr_file):
        speeches_dict = parse_speeches(speeches_file)
        descriptions = parse_description(descr_file)
        speech_objs = create_speech_objects(speeches_dict, descriptions)
        all_speech_objects.update(speech_objs)
    else:
        print(f"Missing speeches or description file for session {session_str}")
    
    # Process bigram score file for this session
    speakers = parse_speaker_map(speaker_map_file)
    speeches_dict = parse_speeches(speeches_file)
    descriptions = parse_description(descr_file)
    session_speeches = create_speech_objects(speeches_dict, descriptions)
    
    # Build a mapping from speech_id to Speaker for this session.
    speech_to_speaker = {}
    for sp in speakers.values():
        for s_id in sp.speeches:
            speech_to_speaker[s_id] = sp


# Build the corpus from all speeches and train a Word2Vec model
corpus = build_corpus(all_speech_objects)

# Generate bigrams using gensim's Phrases
print("Generating Bigrams...")
phrases = Phrases(corpus, min_count=5, threshold=10)  # Tune min_count and threshold as needed
bigram_transformer = Phraser(phrases)
corpus_bigrams = [bigram_transformer[sentence] for sentence in corpus]
print("Bigrams Complete")

# Train the Word2Vec model
print("Training Word2VecModel...")
word2vec_model = train_word2vec(corpus_bigrams)
print("Training Complete")



# Compute similarity scores for sessions 097 to 114 (inclusive)
for session in tqdm(range(97, 115), desc="Computing Similarity Scores:"):
    session_str = f"{session:03d}"  # e.g., "097", "098", ... "114"

    bigram_file = f"C:\\Users\\Maxfield Evers\\Desktop\\Thesis\\Data\\gentzkow_work\\phrase_partisanship\\partisan_phrases_{session_str}.txt"
    # Get top words for this session from the bigram file.
    dem_words, rep_words = get_top_words(bigram_file, n)
    
    # Optional: deduplicate and keep only the top n words.
    dem_words = list(dict.fromkeys(dem_words))[:n]
    rep_words = list(dict.fromkeys(rep_words))[:n]
    print(f"Top Democrat words for session {session_str}:", dem_words)
    print(f"Top Republican words for session {session_str}:", rep_words)

    # Compute session-specific prototype vectors using the global word2vec model.
    D_vector_session = average_vector(word2vec_model, dem_words)
    R_vector_session = average_vector(word2vec_model, rep_words)
    print("D_vector norm:", np.linalg.norm(D_vector_session))
    print("R_vector norm:", np.linalg.norm(R_vector_session))

    # Temporary lists for token-level similarity scores for this session.
    session_D_token_scores = []
    session_R_token_scores = []
    
    # Compute similarity scores for each speech in this session using the session-specific prototypes.
    print(f"Computing similarity scores for session {session_str}...")
    for speech in session_speeches.values():
        # Compute D_sim scores (for Democrat prototype)
        d_raw, d_thresh, d_tokens = compute_similarity_for_speech(speech, word2vec_model, D_vector_session, bigram_transformer, p_threshold)
        # Compute R_sim scores (for Republican prototype)
        r_raw, r_thresh, r_tokens = compute_similarity_for_speech(speech, word2vec_model, R_vector_session, bigram_transformer, p_threshold)

        # Save the raw averages into the speech object.
        speech.D_sim_av = d_raw  # Raw average similarity for D
        speech.R_sim_av = r_raw  # Raw average similarity for R
        # Save the threshold-based averages.
        speech.D_sim = d_thresh
        speech.R_sim = r_thresh
    
        # Append token scores to global lists.
        session_D_token_scores.extend(d_tokens)
        session_R_token_scores.extend(r_tokens)

    # Truncate the token scores to 3 decimals.
    session_D_trunc = [round(score, 2) for score in session_D_token_scores]
    session_R_trunc = [round(score, 2) for score in session_R_token_scores]
    
    # Compute frequency distributions.
    D_freq = Counter(session_D_trunc)
    R_freq = Counter(session_R_trunc)

    # Get all unique truncated scores.
    unique_scores = sorted(set(session_D_trunc).union(set(session_R_trunc)))
    
    # Build the session panel DataFrame.
    print(f"Building .csv for session {session_str}...")
    data = []
    for speech in session_speeches.values():
        if speech.speech_id in speech_to_speaker:
            speaker = speech_to_speaker[speech.speech_id]
            district = f"{speaker.state}-{speaker.district}"
            data.append({
                "date": speech.date,
                "district": district,
                "D_sim_raw": speech.D_sim_av, 
                "D_sim_thresh": speech.D_sim,
                "R_sim_raw": speech.R_sim_av,
                "R_sim_thresh": speech.R_sim,
                "word_count": speech.word_count,
                "session": session_str
            })
                # Create a row for global dataframe
            row = {
                "date": speech.date,
                "district": district,
                "D_sim_raw": speech.D_sim_av, 
                "D_sim_thresh": speech.D_sim,
                "R_sim_raw": speech.R_sim_av,
                "R_sim_thresh": speech.R_sim,
                "word_count": speech.word_count,
                "session": session_str
            }
            rows.append(row)
    session_df = pd.DataFrame(data)
    session_df['date'] = pd.to_datetime(session_df['date'], format='%Y%m%d')
    session_dfs.append(session_df)
    
    # Save the session-specific panel to CSV.
    session_csv = f"C:\\Users\\Maxfield Evers\\Desktop\\Thesis\\Data\\Polarization_Data\\panel_session_{session_str}_n{n}.csv"
    session_df.to_csv(session_csv, index=False)
    print(f"Saved session {session_str} panel data to {session_csv}")

    df = pd.DataFrame(rows)
    
    # Build a DataFrame for the scores distribution (only need one since this is scores on a bigram level).
    dist_rows = []
    for score in unique_scores:
        dist_rows.append({
            "similarity_score": score,
            "D_freq": D_freq.get(score, 0),
            "R_freq": R_freq.get(score, 0)
        })
    df_scores = pd.DataFrame(dist_rows)
    
    # Save to an Excel file specific to this session.
    output_filename = f"C:\\Users\\Maxfield Evers\\Desktop\\Thesis\\Data\\Polarization_Data\\similarity_score_distribution_session_{session_str}_n_{n}.csv"
    df_scores.to_csv(output_filename, index=False)
    print(f"Session {session_str} processed and saved to {output_filename}")



# Combine all sessions into a single panel file.
if session_dfs:
    combined_panel = pd.concat(session_dfs, ignore_index=True)
    combined_csv = f"C:\\Users\\Maxfield Evers\\Desktop\\Thesis\\Data\\Polarization_Data\\congressional_panel_all_sessions_n{n}.csv"
    combined_panel.to_csv(combined_csv, index=False)
    print(f"Saved combined panel data to {combined_csv}")
else:
    print("No session data available.")
    
print("Global panel dataset created and saved as 'congressional_panel_data.csv'")

Building corpus: 100%|███████████████████████████████████████████████| 3866165/3866165 [05:32<00:00, 11635.25it/s]


Generating Bigrams...
Bigrams Complete
Training Word2VecModel...
Training Complete


Computing Similarity Scores::   0%|                                                        | 0/18 [00:00<?, ?it/s]

Top Democrat words for session 097: ['interest_rate', 'high_interest', 'feder_reserv', 'human_right', 'administr_propos', 'el_salvador', 'reserv_board', 'school_lunch', 'oil_compani', 'nuclear_war', 'unemploy_rate', 'minimum_benefit', 'base_mode', 'senior_citizen', 'million_american', 'secur_benefit', 'mx_missil', 'lunch_program', 'saudi_arabia', 'defens_spend', 'nurs_home', 'oil_gas', 'econom_polici', 'farm_incom', 'safeti_net', 'increas_defens', 'budget_propos', 'econom_assumpt', 'make_sens', 'great_depress', 'monetari_polici', 'billion_dollar', 'world_war', 'equal_right', 'medic_care', 'militari_budget', 'habea_corpus', 'vietnam_veteran', 'organ_crime', 'persian_gulf', 'peopl_countri', 'tax_expenditur', 'militari_aid', 'dens_pack', 'war_ii', 'nation_guard', 'militari_spend', 'billion_deficit', 'defens_budget', 'nuclear_arm', 'unemploy_benefit', 'administr_budget', 'arm_control', 'wast_fraud', 'loan_program', 'sinc_great', 'oil_industri', 'tax_break', 'farmer_home', 'hazard_wast', 'i

Computing Similarity Scores::   6%|██▌                                          | 1/18 [06:25<1:49:14, 385.57s/it]

...

Computing Similarity Scores::  94%|█████████████████████████████████████████▌  | 17/18 [1:28:33<05:08, 308.46s/it]

Session 113 processed and saved to C:\Users\Maxfield Evers\Desktop\Thesis\Data\Polarization_Data\similarity_score_distribution_session_113.csv
Top Democrat words for session 114: ['homeland_secur', 'climat_chang', 'gun_violenc', 'african_american', 'vote_right', 'public_health', 'depart_homeland', 'plan_parenthood', 'afford_care', 'puerto_rico', 'right_act', 'background_check', 'care_act', 'middl_class', 'univers_phoenix', 'student_loan', 'mass_shoot', 'reauthor_exportimport', 'minimum_wage', 'year_ago', 'immigr_reform', 'million_american', 'tax_break', 'tar_sand', 'moment_silenc', 'come_togeth', 'reserv_balanc', 'fossil_fuel', 'fund_depart', 'buy_gun', 'govern_shutdown', 'boko_haram', 'assault_weapon', 'tax_credit', 'energi_effici', 'sea_level', 'coast_guard', 'attorney_general', 'special_interest', 'clean_energi', 'opioid_addict', 'first_respond', 'fund_bill', 'right_movement', 'health_safeti', 'million_peopl', 'san_antonio', 'transport_infrastructur', 'gi_bill', 'public_servic', 'he

Computing Similarity Scores:: 100%|████████████████████████████████████████████| 18/18 [1:33:38<00:00, 312.14s/it]

Session 114 processed and saved to C:\Users\Maxfield Evers\Desktop\Thesis\Data\Polarization_Data\similarity_score_distribution_session_114.csv


Saved combined panel data to C:\Users\Maxfield Evers\Desktop\Thesis\Data\Polarization_Data\congressional_panel_all_sessions_n100.csv
Global panel dataset created and saved as 'congressional_panel_data.csv'
